#### Download files from url

- https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
- Download 2023's Yellow Taxi trip records (parquet files)

In [2]:
import os
os.getcwd()

'/Users/reshma/AI/MLOPS Project/taxi_demand_predictor/notebooks'

In [3]:
import requests
from bs4 import BeautifulSoup
import re

In [4]:
url = 'https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page'
resp_url = requests.get(url)

if resp_url.status_code == 200 :
  print(resp_url.status_code)

200


In [6]:
soup = BeautifulSoup(resp_url.text,"html.parser")
yellow_a_tags = soup.find_all(title="Yellow Taxi Trip Records",href=re.compile("2023"))
for link in yellow_a_tags:
    yellow_href = link.get('href')
    
    #Getting an extra %20 at the end of file name due to space in the original a tag on the website
    yellow_href = yellow_href.strip() 
    
    resp_file = requests.get(yellow_href)
    file_name = resp_file.url.split('/')[4]
    %cd '/Users/reshma/AI/MLOPS Project/taxi_demand_predictor/data'
    with open(file_name,'wb') as f:
        print("Downloading file : ",file_name)
        f.write(resp_file.content)

/Users/reshma/AI/MLOPS Project/taxi_demand_predictor/data
/Users/reshma/AI/MLOPS Project/taxi_demand_predictor/data
/Users/reshma/AI/MLOPS Project/taxi_demand_predictor/data
/Users/reshma/AI/MLOPS Project/taxi_demand_predictor/data
/Users/reshma/AI/MLOPS Project/taxi_demand_predictor/data
/Users/reshma/AI/MLOPS Project/taxi_demand_predictor/data
